In [4]:
import pandas as pd
from scipy.optimize import linprog
import numpy as np
from gurobipy import GRB
import gurobipy as gb
from gurobipy import Model, GRB, quicksum


In [3]:
import gurobipy

In [2]:
pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 3.7 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
model = gb.Model("Optimization Problem")

Restricted license - for non-production use only - expires 2025-11-24


In [7]:
df_direct_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Direct_Production_Facilities.csv')
df_transship_production_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Production_Facilities.csv')
df_transship_distribution_capacity = pd.read_csv('/Users/youziya/Downloads/Capacity_for_Transship_Distribution_Centers.csv')
df_cost_production_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Transshipment.csv')
df_cost_production_to_transshipment = pd.read_csv('/Users/youziya/Downloads/Cost_Production_to_Refinement.csv')
df_cost_transshipment_to_refinement = pd.read_csv('/Users/youziya/Downloads/Cost_Transshipment_to_Refinement.csv')
df_refinement_demand = pd.read_csv('/Users/youziya/Downloads/Refinement_Demand.csv')


In [8]:
dfs = {
    'Direct Production Capacity': df_direct_production_capacity,
    'Transship Production Capacity': df_transship_production_capacity,
    'Transship Distribution Capacity': df_transship_distribution_capacity,
    'Cost Production to Refinement': df_cost_production_to_refinement,
    'Cost Production to Transshipment': df_cost_production_to_transshipment,
    'Cost Transshipment to Refinement': df_cost_transshipment_to_refinement,
    'Refinement Demand': df_refinement_demand
}

dfs


{'Direct Production Capacity':     ProductionFacility  Capacity
 0                    1       462
 1                    2       103
 2                    3       460
 3                    4       325
 4                    5       227
 5                    6       217
 6                    7       205
 7                    8       521
 8                    9       548
 9                   10       191
 10                  11       361
 11                  12       411
 12                  13       104
 13                  14       155
 14                  15       285
 15                  16       109
 16                  17       422
 17                  18       438
 18                  19       501
 19                  20       139
 20                  21       462
 21                  22       504
 22                  23       106
 23                  24       132
 24                  25       298,
 'Transship Production Capacity':     ProductionFacility  Capacity
 0                

In [ ]:
directproduction_capacity = [[
462,
460,
325,
227,
217,
205,
521,
548,
191,
361,
411,
104,
155,
285,
109,
422,
438,
501,
139,
462,
504,
106,
132,
298]]

tranship_capacity = [[
374
444
395
245
378
408
435
175
415
503
184
297
450
169
365]]
trans_capacity = [[1317, 1453]]

# Create the a single class of decision variables where
# From = {𝑫𝒂,𝑯𝒐} and To = {𝑪𝒉,𝑳𝑨,𝑺𝑭,𝑵𝒀}.
x = model.addVars(2, 4, lb=0, vtype=GRB.CONTINUOUS, name="Source Nodes")
# From = {𝑪𝒉,𝑳𝑨} and To = {𝑺𝑭,𝑵𝒀}.
y = model.addVars(2, 2, lb=0, vtype=GRB.CONTINUOUS, name="Transshipment Nodes")


In [ ]:
# Add the supply constraints from source nodes
model.addConstr(gb.quicksum(x[0,j] for j in range(4)) <= 200, name="Supply Constraint 1")
model.addConstr(gb.quicksum(x[1,j] for j in range(4)) <= 160, name="Supply Constraint 2")

In [ ]:

# Adjusting the capacity constraints for direct production facilities
A_direct_capacity_revised = np.zeros((n_direct_facilities, len(c)))
offset = 0
for i in range(n_direct_facilities):
    A_direct_capacity_revised[i, offset:offset+n_refinement_centers] = 1
    offset += n_refinement_centers

# Adjusting the capacity constraints for transshipment production facilities
A_transship_prod_capacity_revised = np.zeros((n_transship_facilities, len(c)))
offset = n_direct_facilities * n_refinement_centers
for i in range(n_transship_facilities):
    A_transship_prod_capacity_revised[i, offset:offset+n_transship_centers] = 1
    offset += n_transship_centers

# Adjusting the capacity constraints for transshipment hubs
A_transship_hub_capacity_revised = np.zeros((n_transship_centers, len(c)))
offset = n_direct_facilities * n_refinement_centers + n_transship_facilities * n_transship_centers
for k in range(n_transship_centers):
    A_transship_hub_capacity_revised[k, offset:offset+n_refinement_centers] = 1
    offset += n_refinement_centers

# Adjusting the demand constraints at each refinement center
A_demand_revised = np.zeros((n_refinement_centers, len(c)))
for j in range(n_refinement_centers):
    # Direct production facilities to refinement centers
    for i in range(n_direct_facilities):
        A_demand_revised[j, i * n_refinement_centers + j] = 1
    # Transshipment hubs to refinement centers
    for k in range(n_transship_centers):
        A_demand_revised[j, n_direct_facilities * n_refinement_centers + n_transship_facilities * n_transship_centers + k * n_refinement_centers + j] = 1

# Combine all constraints
A_revised = np.vstack([A_direct_capacity_revised, A_transship_prod_capacity_revised, A_transship_hub_capacity_revised, A_demand_revised])
b_revised = np.hstack([b_direct_capacity, b_transship_prod_capacity, b_transship_hub_capacity, b_demand])

# Solve the linear programming problem again
result_revised = linprog(c, A_ub=A_revised, b_ub=b_revised, bounds=bounds, method='highs')

result_revised.success, result_revised.fun, result_revised.x

